In [1]:
import numpy as np
from scipy.fftpack import irfft, rfftfreq
from scipy.signal import windows
from matplotlib import pyplot as plt
from ipywidgets import interact

# Parámetros generales
sample_rate = 1000  # Hz
duration = 2        # segundos
N = sample_rate * duration
frequencies = rfftfreq(N, d=1/sample_rate)

# 1. Crear espectro escalón (bandstop con corte rectangular)
def create_step_bandstop(f_stop_start=100, f_stop_end=200):
    spectrum = np.ones_like(frequencies)
    # Hacer 0 entre f_stop_start y f_stop_end (bandstop rectangular)
    spectrum[(frequencies >= f_stop_start) & (frequencies <= f_stop_end)] = 0
    return spectrum

# 2. Aplicar ventana y transformar a dominio del tiempo
def generate_signal(f_stop_start=100, f_stop_end=200):
    spectrum = create_step_bandstop(f_stop_start, f_stop_end)
    window = windows.hamming(len(spectrum))  # Enventanado
    spectrum_windowed = spectrum * window
    signal = irfft(spectrum_windowed)
    return signal, spectrum_windowed

# 3. Ecualizador de una banda
@interact(fc=(10, 490, 10), gain_db=(-20, 20, 1))
def equalizer(fc=100, gain_db=0):
    f_stop_start = 200
    f_stop_end = 300
    signal, spectrum = generate_signal(f_stop_start, f_stop_end)

    # Crear filtro ecualizador
    eq_spectrum = np.ones_like(spectrum)
    bw = 20  # ancho de banda fijo de 20 Hz
    idx = (frequencies >= fc - bw) & (frequencies <= fc + bw)
    eq_spectrum[idx] *= 10**(gain_db / 20)  # aplicar ganancia en dB

    # Aplicar ecualización en frecuencia
    spectrum_eq = spectrum * eq_spectrum
    signal_eq = irfft(spectrum_eq)

    # Gráficas
    plt.figure(figsize=(16, 6))

    plt.subplot(2, 1, 1)
    plt.title("Señal en el tiempo (ecualizada)")
    plt.plot(signal_eq)
    plt.xlabel("Muestras")
    plt.ylabel("Amplitud")

    plt.subplot(2, 1, 2)
    plt.title("Espectro en frecuencia (después del enventanado y ecualización)")
    plt.plot(frequencies, np.abs(spectrum_eq))
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Magnitud")
    plt.grid()
    plt.tight_layout()
    plt.show()


interactive(children=(IntSlider(value=100, description='fc', max=490, min=10, step=10), IntSlider(value=0, des…